[LRU Cache](https://zhuanlan.zhihu.com/p/35175401)

LRU算法管理缓存数据（Least Recently Used,最近最久未使用法）
最近使用的页面数据会在未来一段时期内仍然被使用,已经很久没有使用的页面很有可能在未来较长的一段时间内仍然不会被使用
https://www.jianshu.com/p/74a4efacb0a7
一般是基于双向链表实现，但是考虑到避免在内存中大量拷贝操作，将数据存到hash map中

In [19]:

def cache(func):
    data={}
    def wrapper(n):
        if n not in data:
            data[n]=func(n)
        return data[n]
    return wrapper

"""
cache before:1.1854779720306396
cache after:0.00016617774963378906
"""           
@cache
def fib(n):
    if n<=2:
        return 1
    return fib(n-2)+fib(n-1)


import time
t=time.time()
fib(33)
time.time()-t

0.00016617774963378906

In [4]:
# LRU cache
from functools import wraps
import json

class Node():
    def __init__(self,prev=None,next=None,key=None,value=None):
        self.prev,self.next,self.key,self.value=prev,next,key,value
        

class DoubleDirectionLink():
    def __init__(self):
        node=Node()
        node.prev,node.next=node,node
        self.rootnode=node
        self.cache = {}
        
    def headnode(self):
        return self.rootnode.next
    
    @property
    def tailnode(self):
        return self.rootnode.prev
    
    def remove(self,node):
        if node is self.rootnode:
            return
        else:
            node.prev.next=node.next
            node.next.prev=node.prev
            
    def append(self,node):
        tailnode=self.tailnode()
        tailnode.next=node
        node.next=self.rootnode
        node.prev=tailnode
        self.rootnode.prev=node
    
    def append2head(self, node):
        node.next = self.rootnode.next
        node.prev = self.rootnode
        self.rootnode.next.prev = node
        self.rootnode.next = node
    
    def node2head(self, node):
        # 将节点移动到队首
        if self.rootnode.next == node:
            return
        self.remove(node)
        self.append2head(node)
        
    def print_value(self):
        values = []
        node = self.rootnode.next
        if not node:
            return
        values.append(node.value)
        while node.next and node.next != self.rootnode:
            values.append(node.next.value)
            node = node.next
        print(values)
        
    
class LRUCache():
    def __init__(self,capacity=5):
        self.capacity=capacity
        self.cache={}
        self.access=DoubleDirectionLink()
        
    def __call__(self,func):
        @wraps(func)
        def wrapper(*args,**kargs):
            # 从cache中获取缓存，如果存在则直接返回缓存结果，并且将此缓存移到队头
            # 如果不存在，则调用func，将计算结果返回，并且将结果插入到队头，判断是否超出容量，超出则淘汰队尾节点
            key = ''.join(args) + json.dumps(kargs)
            node = self.cache.get(key)
            print(key, node)

            if node:
                self.access.node2head(node)
            else:
                value = func(*args,**kargs)
                node = Node(key=key, value=value)
                self.access.append2head(node)
                self.cache.update({key: node})
                if len(self.cache) > self.capacity:
                    self.cache.pop(self.access.tailnode.key)
                    self.access.remove(self.access.tailnode)
            return node.value
        self.access.print_value()
        return wrapper
        
        
# 测试
from string import hexdigits
import random

data = [random.choice(hexdigits) for _ in range(10)]
print(data)

@LRUCache()
def test_func(char):
    value = char + '_lru'
    return value


for char in data:
    test_func(char) 
LRUCache().access.print_value()

['D', 'a', '8', 'c', 'D', 'd', 'B', 'C', 'c', '5']
[None]
D{} None
a{} None
8{} None
c{} None
D{} <__main__.Node object at 0x10a74bf98>
d{} None
B{} None
C{} None
c{} <__main__.Node object at 0x10a75b278>
5{} None
[None]


In [34]:
pow(4, 16)

4294967296